<a href="https://colab.research.google.com/github/lucasMenotti/EmotiBot-imers-o_dev-/blob/main/EmotiBot_Escolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import display, HTML

display(HTML("""
<style>
  .capa-projeto {
    background: linear-gradient(135deg, #E36937, #F5A623);
    color: white;
    padding: 60px 40px;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    border-radius: 20px;
    box-shadow: 0 8px 20px rgba(0,0,0,0.3);
    text-align: center;
  }
  .capa-projeto h1 {
    font-size: 3.5em;
    margin-bottom: 0.2em;
    text-shadow: 2px 2px 8px rgba(0,0,0,0.4);
  }
  .capa-projeto h2 {
    font-weight: 300;
    font-size: 1.7em;
    margin-top: 0;
    margin-bottom: 1em;
    font-style: italic;
  }
  .capa-projeto p {
    font-size: 1.3em;
    margin-bottom: 0.5em;
  }
  .capa-projeto small {
    font-size: 0.95em;
    color: #ffd9b3;
  }
</style>

<div class="capa-projeto">
  <h1>🤖 EmotiBot</h1>
  <h2>Seu Amigo Virtual de Apoio Emocional</h2>
  <p><strong>Projeto para:</strong> Suporte a jovens com ansiedade escolar</p>
  <p><strong>Objetivo:</strong> Ajudar, escutar e sugerir pausas para autocuidado</p>
  <small>Apresentado por: Lucas</small>
</div>
"""))


In [17]:
### Apresentação do Projeto EmotiBot

### Um dado chocante:

#- Cerca de **30% dos jovens brasileiros** enfrentam ansiedade escolar severa, sem apoio emocional adequado.

### Quem sofre?

#- Estudantes entre 12 e 25 anos, que sentem medo, estresse e falta de foco nas tarefas, prejudicando seu desenvolvimento.

### O que é o EmotiBot?

#- Um **amigo virtual empático**, um **guia acolhedor** e um **especialista em autocuidado** na palma da sua mão.
#- Ele escuta, compreende e ajuda a gerenciar a ansiedade com dicas, exercícios simples e a sugestão de pausas para autocuidado.

### Demonstração interativa

#- Vamos ver o EmotiBot em ação no Google Colab! Você pode conversar, pedir ajuda e muito mais!

### Visão de impacto

# Esse chatbot pode impactar milhões de jovens que hoje não têm acesso a apoio emocional, trazendo conforto, foco e bem-estar para suas vidas escolares.


In [7]:
%pip -q install google-genai google-adk

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

from IPython.display import display, Markdown
import textwrap
import warnings
from datetime import date

warnings.filterwarnings("ignore")


In [8]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response


In [9]:
agente_ouvinte = Agent(
    name="ouvinte_emocional",
    model="gemini-2.0-flash",
    instruction="""
        Você é um ouvinte empático. Ao receber uma mensagem do jovem, identifique como ele está se sentindo
        (ansioso, triste, estressado, feliz) e formule um pequeno resumo emocional.
        Não resolva ainda, apenas demonstre que compreendeu.
    """,
    description="Agente que escuta e identifica emoções"
)


In [10]:
agente_apoio = Agent(
    name="apoio_emocional",
    model="gemini-2.0-flash",
    instruction="""
        Você é um amigo digital com empatia. Baseado no resumo emocional, ofereça:
        - Uma resposta acolhedora e tranquila
        - Um exercício de respiração ou relaxamento
        - Uma sugestão leve para se sentir melhor
    """,
    description="Agente de apoio emocional"
)


In [20]:
agente_conselheiro = Agent(
    name="conselheiro_escolar",
    model="gemini-2.0-flash",
    instruction="""
        Você é um conselheiro educacional e emocional. Dê um conselho curto e prático para um jovem lidando com ansiedade escolar,
        como dicas de estudo, organização, pausas, ou autoestima de acordo com a situação que ele esta passando.
    """,
    description="Agente conselheiro"
)


In [21]:
agente_positivo = Agent(
    name="reforco_positivo",
    model="gemini-2.0-flash",
    instruction="""
        Você é um gerador de reforços positivos. Diga uma frase motivacional, curta e inspiradora,
        que possa deixar o jovem com esperança no final da conversa.
    """,
    description="Agente de reforço emocional positivo"
)


In [23]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
print(f"🤝 Chatbot EmotiBot Escolar — {data_de_hoje}")
desabafo = input("Digite a seguir uma dúvida, desabafo ou situação que você esteja passando nos estudos : ")


# Etapa 1: Ouvinte
emocao = call_agent(agente_ouvinte, desabafo)
display(Markdown("### 👂 Ouvinte:\n" + emocao))

# Etapa 2: Apoio emocional
apoio = call_agent(agente_apoio, emocao)
display(Markdown("### 🧘 Apoio Emocional:\n" + apoio))

# Etapa 3: Conselheiro
conselho = call_agent(agente_conselheiro, emocao)
display(Markdown("### 🎓 Conselho:\n" + conselho))

# Etapa 4: Reforço Positivo
reforco = call_agent(agente_positivo, emocao)
display(Markdown("### 🌟 Palavra Final:\n" + reforco))



🤝 Chatbot EmotiBot Escolar — 17/05/2025
Digite a seguir uma dúvida, desabafo ou situação que você esteja passando nos estudos : estou me sentindo inutil, estudei tanto para as provas e no final fui muito mal nelas, estou me sentindo incapaz de tudo


### 👂 Ouvinte:
Entendo. Parece que você está se sentindo muito inútil e incapaz no momento, especialmente após ter se dedicado tanto aos estudos e não ter obtido os resultados esperados nas provas. É como se todo o seu esforço não tivesse valido a pena, e isso está te deixando bastante frustrado e desanimado.


### 🧘 Apoio Emocional:
Ei, sinto muito que você esteja se sentindo assim. É muito frustrante quando a gente se dedica e não vê o resultado que espera. Mas olha, essa sensação não define você, tá?

Que tal a gente fazer um exercício rapidinho? Respire fundo pelo nariz, contando até quatro. Sinta o ar enchendo seus pulmões. Agora, solte o ar lentamente pela boca, contando até seis. Repita isso algumas vezes. Essa respiração pode te ajudar a relaxar um pouco e clarear a mente.

E ei, que tal fazer algo que você gosta muito agora? Assistir um filme, ouvir sua música favorita, ou até mesmo dar uma volta lá fora. Às vezes, um pequeno momento de prazer pode fazer toda a diferença. 😊



### 🎓 Conselho:
Ei, campeão! Olha, sei bem como é essa sensação de ralar e não ver o resultado que esperava. Mas deixa eu te lembrar de uma coisa: você não é uma nota!

**Primeiro:** Que tal a gente dar um passo atrás e analisar como você está estudando? Às vezes, não é a quantidade, mas a qualidade do estudo que faz a diferença. Experimente técnicas como a de Pomodoro (estudar por 25 minutos e fazer pausas de 5 minutos) ou mapas mentais para organizar as ideias.

**Segundo:** Seja gentil com você mesmo. Você se dedicou, e isso já é uma vitória! Reconheça o esforço que você fez e se recompense por isso. Assista um filme, saia com amigos, faça algo que te dê prazer.

**Terceiro:** Que tal a gente focar em pequenas metas? Em vez de pensar em tirar 10 na próxima prova, que tal se desafiar a entender um conceito que você tem dificuldade por dia? Pequenos progressos te darão mais confiança e te motivarão a seguir em frente.

Lembre-se: você é capaz, inteligente e tem muito potencial. Acredite em si mesmo e não deixe que uma nota te defina! 😉



### 🌟 Palavra Final:
"Acredite no seu potencial, pois a cada desafio superado, você se torna mais forte e resiliente!"

